### 💡Desafio: Relatórios Avançados e Dashboard de Vendas
Agora que você já armazenou os dados no SQLite, o desafio será consultar e analisar os dados diretamente via SQL, além de criar um dashboard interativo para visualização.

📌 Parte 1 - Consultas SQL Avançadas
1. Clientes mais frequentes 🔁
  a. Liste os 10 clientes que mais compraram (quantidade de pedidos realizados).
2. Produtos mais rentáveis 💰
  a. Encontre os 5 produtos que mais geraram receita total.
3. Média de valor gasto por categoria 📊
  a. Liste as categorias de produtos com a média de valor gasto por pedido.
4. Comparação de faturamento 📅
  a. Compare o faturamento entre o primeiro semestre (Jan-Jun) e o segundo semestre (Jul-Dez) de 2024.

📌 Parte 2 - Dashboard Interativo com Streamlit
Agora, crie um dashboard interativo usando Streamlit para visualizar os dados de forma dinâmica.

1. 📌 Componentes do Dashboard:
  a. Filtro por mês 📆 (para visualizar as vendas de um período específico).
  b. Gráficos interativos 📊:
    I. Gráfico de barras com os produtos mais vendidos.
    II. Gráfico de linha mostrando o faturamento mensal.
    III. Tabela interativa com os clientes VIPs.

Data Analysis 🔎⚙️🧩

In [30]:
# Importando as bibliotecas
import pandas as pd
import sqlite3
import streamlit as st

conn = sqlite3.connect('salesEcommerce.db') 
cursor = conn.cursor()

# Criando dicionário de queries
dicQueries = {
    # Query com 10 clientes que mais compraram
    "queryTopCustomers": ''' 
        SELECT
            customer,
            SUM(sales_quantity) AS sum_quantity
        FROM
            sales
        GROUP BY
            customer
        ORDER BY
            SUM(sales_quantity) DESC
        LIMIT 10
''',
# Query com 5 produtos mais rentáveis
    "queryTopProducts": '''
        SELECT
            product,
            SUM(total_price) AS total_revenue
        FROM
            sales
        GROUP BY
            product
        ORDER BY
            SUM(total_price) DESC
        LIMIT 5
''',
# Média de valor gasto por categoria
    "queryAvgPriceCategory": '''
        SELECT
            category,
            ROUND(AVG(total_price),2) AS avg_price
        FROM
            sales
        GROUP BY
            category
        ORDER BY
            AVG(total_price) DESC
''',
# Comparação de faturamento primeiro semestre (Jan-Jun) e o segundo semestre (Jul-Dez) de 2024.
    "queryHalfYear": '''
        SELECT
            CASE 
                WHEN STRFTIME('%m', sales_date) BETWEEN '01' AND '06' THEN 'first half-year'
                WHEN STRFTIME('%m', sales_date) BETWEEN '07' AND '12' THEN 'second half-year'
            END as half_year,
            ROUND(SUM(total_price),2) AS total_revenue
        FROM
            sales
        WHERE
            STRFTIME('%Y', sales_date) = '2024'
        GROUP BY
            half_year
        ORDER BY
            half_year
        '''
}

In [31]:
# Gerador de queries
def runQuery(queryKey, params=()):
    query = dicQueries[queryKey]
    return pd.read_sql(query,conn, params=params)

dfTopCustomers = runQuery("queryTopCustomers")
dfTopProducts = runQuery("queryTopProducts")
dfAvgPriceCategory = runQuery("queryAvgPriceCategory")
dfHalfYear = runQuery("queryHalfYear")

# Exibindo os resultados
print("Top 10 Clientes:")
print(dfTopCustomers)

print("\nTop 5 Produtos mais rentáveis:")
print(dfTopProducts)

print("\nMédia de valor gasto por categoria:")
print(dfAvgPriceCategory)

print("\nComparação de faturamento Jan-Jun e Jul-Dez:")
print(dfHalfYear)

Top 10 Clientes:
      customer  sum_quantity
0         José           166
1      Daniela           145
2      Mirella           142
3        Josué           139
4        Renan           136
5         Igor           134
6  Maria Luísa           133
7       Allana           131
8        Isaac           129
9         Ayla           129

Top 5 Produtos mais rentáveis:
                product  total_revenue
0     Mochila Esportiva       94488.69
1               Mindset       94163.18
2  Geladeira Frost Free       93767.95
3    Cama para Cachorro       87807.27
4          Câmera de Ré       87708.74

Média de valor gasto por categoria:
            category  avg_price
0           Esportes    2388.76
1             Roupas    2259.86
2             Moveis    2240.68
3             Livros    2220.29
4              Games    2193.02
5         Brinquedos    2148.88
6         Automotivo    2128.71
7        Ferramentas    2112.79
8   Eletrodomesticos    2107.39
9             Beleza    2078.70
10      C

Data Visualization 📊📈📉

In [32]:
st.title("📊 Sales Dashboard - E-commerce")

2025-03-07 15:22:02.473 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-07 15:22:02.485 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [33]:
# Filtro por mês(visualizar vendas de um período específico).
monthFilter = st.selectbox("Selecione um mês:", ["Todos"] + [f"{m:02d}/2024" for m in range(1, 13)])

2025-03-07 15:22:02.689 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-07 15:22:02.697 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-07 15:22:02.701 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-07 15:22:02.709 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-07 15:22:02.717 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-07 15:22:02.722 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [34]:
dicQueriesViz = {
# Gráfico de barras com os produtos mais vendidos.
"queryBarTopProducts": '''
SELECT
    product,
    SUM(sales_quantity) AS total_quantity
FROM
    sales
{whereClause}
GROUP BY
    product
ORDER BY
    SUM(sales_quantity) DESC
LIMIT 10
''',
# Gráfico de linha mostrando o faturamento mensal.
"queryLineMonthRevenue": '''
SELECT
    STRFTIME('%m', sales_date) as month,
    SUM(total_price) AS total_revenue
FROM
    sales
{whereClause}
GROUP BY
    STRFTIME('%m', sales_date)
ORDER BY
    STRFTIME('%m', sales_date)
''',
# Tabela interativa com os clientes VIPs.
"queryVipCustomers": '''
SELECT
    customer,
    SUM(sales_quantity) AS total_quantity,
    SUM(total_price) AS total_amount
FROM
    sales
{whereClause}
GROUP BY
    customer
ORDER BY
    SUM(total_price) DESC,
    SUM(sales_quantity) DESC,
    customer
LIMIT 10
'''
}

In [35]:
# Ajustando a cláusula WHERE
whereClause = ""
params = ()
if monthFilter != "Todos":
    selectMonth = monthFilter.split("/")[0]
    whereClause = "WHERE STRFTIME('%m', sales_date) = ? AND STRFTIME('%Y', sales_date) = '2024'"
    params = (selectMonth,)

In [36]:
# Função para executar queries de visualização
def runQueryViz(queryKey):
    query = dicQueriesViz[queryKey].format(whereClause=whereClause)
    return pd.read_sql(query, conn, params=params)

In [37]:
# Exibindo gráfico de barras com os produtos mais vendidos
st.write("Gráfico de barras com os produtos mais vendidos.")
dfBarTopProducts = runQueryViz("queryBarTopProducts")
if not dfBarTopProducts.empty:
    st.bar_chart(dfBarTopProducts.set_index('product'), x_label="Products", y_label="Quantity")
else:
    st.write("Nenhum dado disponível para o período selecionado.")

2025-03-07 15:22:03.372 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-07 15:22:03.374 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-07 15:22:03.379 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-07 15:22:03.381 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-07 15:22:04.637 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-07 15:22:04.637 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [38]:
# Gráfico de linha mostrando o faturamento mensal.
st.write("Gráfico de linha faturamento mensal.")
dfLineMonthRevenue = runQueryViz("queryLineMonthRevenue")
if not dfLineMonthRevenue.empty:
    st.line_chart(dfLineMonthRevenue.set_index('month'), x_label="Months", y_label="Revenue")
else:
    st.write("Nenhum dado disponível para o período selecionado.")

2025-03-07 15:22:04.671 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-07 15:22:04.673 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-07 15:22:04.675 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-07 15:22:04.675 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-07 15:22:04.926 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-07 15:22:04.928 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [39]:
# Tabela interativa com os clientes VIPs.
st.write("Tabela interativa com os clientes VIPs")
dfVipCustomers = runQueryViz("queryVipCustomers")
if not dfVipCustomers.empty:
    st.dataframe(dfVipCustomers.style.highlight_max(subset=['total_quantity', 'total_amount']), hide_index=True)
else:
    st.write("Nenhum dado disponível para o período selecionado.")

2025-03-07 15:22:05.051 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-07 15:22:05.053 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-07 15:22:05.055 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-07 15:22:05.057 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-07 15:22:05.086 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-07 15:22:05.291 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-07 15:22:05.295 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [40]:
# Fechando a conexão com o banco de dados
conn.close()